- 새로운 가상환경 생성
conda create --name codyssey pandas
- 가상 환경 실행
conda activate codyssey
- 영화 개수, 배급사 수, 감독 수, 출연진 수, 장르 수를 파악

In [156]:
import pandas as pd

In [157]:
# 데이터 읽기
data = pd.read_csv('prob-0101.csv')

In [158]:
# 각 항목 계산
num_movies = len(data)  # 영화 개수
num_distributors = data['배급사'].nunique()  # 배급사 수 (중복 제거)
num_directors = data['감독'].nunique()  # 감독 수 (중복 제거)
num_actors = data['출연진'].apply(lambda x: len(x.split(','))).sum()  # 출연진 수 (쉼표로 구분 후 합산)
num_genres = data['장르'].apply(lambda x: len(x.split(','))).sum()  # 장르 수 (쉼표로 구분 후 합산)

In [159]:
print(f"영화 개수: {num_movies}")
print(f"배급사 수: {num_distributors}")
print(f"감독 수: {num_directors}")
print(f"출연진 수: {num_actors}")
print(f"장르 수: {num_genres}")

영화 개수: 59
배급사 수: 52
감독 수: 58
출연진 수: 275
장르 수: 68


- 함수 작성
csv 파일을 읽어 DataFrame으로 저장한 data 변수를 파라메터로 받음
배급사가 한개 이상이 경우 ', '로 구별
감독은 여러명인 경우가 없음
출연진은 한개 이상의 경우 ', '로 구별
장르는 한개 이상의 경우 '·'나 '/'를 ','로 변경하고 
구별하는게 어떤경우는 ', '이지만 장르 경우는 ','로 구별해야 하기도 함. 따라서 정규식 표현으로 구별.

In [160]:
import re

- 정규표현식 r'[,·/]|\s*,\s*'의 의미
[,·/]: ',', '·', '/' 중 하나와 일치
|: 또는
\s*,\s*: 선택적 공백과 쉼표, 그리고 다시 선택적 공백
- 정규식 검증
(참고) https://inpa.tistory.com/entry/%F0%9F%92%BB-%EC%A0%95%EA%B7%9C%EC%8B%9D-%ED%85%8C%EC%8A%A4%ED%8A%B8-%EC%82%AC%EC%9D%B4%ED%8A%B8-%F0%9F%8E%81-%EB%AA%A8%EC%9D%8C
https://regexr.com/
https://regexr.com/8ad0p
(참고) https://yenbook.tistory.com/66
https://www.regextester.com/

def analyze_movie_data(data):
    # 고유한 값 추출
    unique_distributors = set()
    unique_directors = set()
    unique_cast = set()
    unique_genres = set()

    for _, row in data.iterrows():
        # 배급사 분리 및 추가
        distributors = re.split(r'[,·/]|\s*,\s*', row['배급사'])
        # distributors = row['배급사'].split(', ')
        # if len(distributors) > 1:
        #     for distributor in distributors:
        #         unique_distributors.update(distributor)
        # else:
        #     unique_distributors.update(distributors)
        unique_distributors.update(distributors)
        
        # 감독 추가
        directors = re.split(r'[,·/]|\s*,\s*', row['감독'])
        unique_directors.update(directors)

        # 출연진 분리 및 추가
        cast = re.split(r'[,·/]|\s*,\s*', row['출연진'])
        #cast = row['출연진'].split(', ')
        # if len(cast) > 1:
        #     for actor in cast:
        #         unique_cast.update(actor)
        # else:
        #     unique_cast.update(cast)
        unique_cast.update(cast)
        
        # 장르 분리 및 추가
        row['장르'] = row['장르'].replace('·', ', ')
        row['장르'] = row['장르'].replace('/', ', ')
        #genres = row['장르'].split(', ')
        genres = re.split(r'[,·/]|\s*,\s*', row['장르'])
        # print("genres: ", genres)
        unique_genres.update(genres)

    # 결과 계산
    num_movies = len(data)
    num_distributors = len(unique_distributors)
    num_directors = len(unique_directors)
    num_cast = len(unique_cast)
    num_genres = len(unique_genres)
    # print("장르: ", unique_genres)
    return num_movies, num_distributors, num_directors, num_cast, num_genres, unique_genres, unique_cast

In [161]:
def analyze_movie_data(data):
    unique_distributors = set()
    unique_directors = set()
    unique_cast = set()
    unique_genres = set()

    for _, row in data.iterrows():
        unique_distributors.update(re.split(r'[,·/]|\s*,\s*', row['배급사']))
        # print(_, row['감독'])
        # data 원본교체, 이전 data는 prob-0101.csv.old 로 수정
        # 따라서, 감독열에 중복값 있음. 중복 제거가 필요해서 add 함수를 update로 수정
        # add는 단일값만 추가 가능하나, update는 여러 요소를 추가 가능하기 때문문
        # unique_directors.add(row['감독'])
        unique_directors.update(re.split(r'[,·/]|\s*,\s*', row['감독']))
        unique_cast.update(re.split(r'[,·/]|\s,\s', row['출연진']))
        unique_genres.update(re.split(r'[,·/]|\s*,\s*', row['장르']))

    return (
        len(data),
        len(unique_distributors),
        len(unique_directors),
        len(unique_cast),
        len(unique_genres)
    )


In [162]:
# 데이터 분석 실행
result = analyze_movie_data(data)
print(f"영화 개수: {result[0]}")
print(f"배급사 수: {num_distributors}, 배급사 수: {result[1]}")
print(f"감독 수: {num_directors}, 감독 수: {result[2]}")
print(f"출연진 수: {num_actors}, 출연진 수: {result[3]}")
print(f"장르 수: {num_genres}, 장르 수: {result[4]}")

영화 개수: 59
배급사 수: 52, 배급사 수: 55
감독 수: 58, 감독 수: 62
출연진 수: 275, 출연진 수: 268
장르 수: 68, 장르 수: 17


print(f"고유 장르: {result[5]}")
print(f"고유 출연진: {result[6]}")

In [163]:
# 출연진의 중복을 제거하고 고유한 출연진 수를 계산
unique_actors = set()

def add_actors(actors_string):
    if isinstance(actors_string, str):
        actors = [actor.strip() for actor in actors_string.split(',')]
        unique_actors.update(actors)

data['출연진'].apply(add_actors)
num_unique_actors = len(unique_actors)
print("고유 출연진 수:", num_unique_actors)

고유 출연진 수: 262


def calculate_unique_actors(data):
    unique_actors = set()

    def process_actors(actors_string):
        if isinstance(actors_string, str):
            # 정규식을 사용하여 출연진 분리
            actors = re.split(r'[,·/]\s*', actors_string)
            return [actor.strip() for actor in actors]
        return []

    # 각 행의 출연진을 처리하고 고유한 배우 집합에 추가
    for actors_list in data['출연진'].apply(process_actors):
        unique_actors.update(actors_list)

    return len(unique_actors)

# 고유한 출연진 수 계산
num_unique_actors = calculate_unique_actors(data)
print("고유 출연진 수:", num_unique_actors)

In [164]:
num = 0
for actor in unique_actors:
  num += 1
  print (num, actor)

1 장동윤
2 이민정
3 김재화
4 지일주
5 이동휘
6 김상욱
7 고창석
8 김민경
9 박종진
10 진혜원
11 전소니
12 유재인
13 강길우
14 김용준
15 송은지
16 이선균
17 유인수
18 이지용
19 마동석
20 최우준
21 옥자연
22 박유나
23 이범수
24 나현준
25 임지섭
26 공명
27 김동희
28 지연
29 정병훈
30 송덕호
31 박서준
32 이이경
33 권상우
34 안재홍
35 김강우
36 오준택
37 김다현
38 서지안
39 김명임
40 홍준영
41 진선규
42 김정팔
43 이나영
44 조윤서
45 정미미
46 성유빈
47 방진원
48 설유진
49 이현우
50 한가은
51 백수련
52 송보은
53 강진아
54 장영준
55 윤제문
56 유지애
57 심소영
58 김경일
59 한수연
60 김소희
61 신민재
62 채수아
63 이설구
64 이한주
65 하민호
66 김소형
67 전신환
68 윤박
69 양현민
70 박동기
71 심영은
72 한규원
73 안수호
74 정수지
75 이용녀
76 한태경
77 정은채
78 이대연
79 공민정
80 박해수
81 조은지
82 박정철
83 이경준
84 차태현
85 노수산나
86 하성국
87 한승도
88 김준형
89 정승길
90 강태주
91 김승수
92 노지유
93 정다은
94 정서하
95 김준
96 서효림
97 소이
98 황정민
99 조수빈
100 이원석
101 문지인
102 박선호
103 허웅
104 아이유
105 최동구
106 박호산
107 노상현
108 신의현
109 정태우
110 박종환
111 최성원
112 김승윤
113 설인아
114 권민혁
115 이규복
116 조진웅
117 김무열
118 서지후
119 강기영
120 박영훈
121 안미나
122 김종수
123 인내상
124 김선영
125 박미현
126 김민재
127 김정균
128 유연석
129 김선호
130 김우겸
131 정아미
132 최민수
133 오경화
134 서벽준
135 백지혜
136 김명찬
137 박순천
138 정영주
139 장동주
1

** 영화 추천 방식 **
- 개인 취양 장르/배우 로 추천
- 추천 갯수 입력 (3~5개 이하 권장)

In [165]:
import pandas as pd
import random

# CSV 파일 읽기
# data = pd.read_csv('prob-0101.csv')

def recommend_movies(preferred_genre, num_recommendations):
    # 선호하는 장르에 해당하는 영화 필터링
    genre_movies = data[data['장르'].str.contains(preferred_genre, case=False, na=False)]
    
    # 추천할 영화 수가 해당 장르의 영화 수보다 많으면 조정
    num_recommendations = min(num_recommendations, len(genre_movies))
    
    # 랜덤하게 영화 선택
    recommended_movies = genre_movies.sample(n=num_recommendations)
    
    return recommended_movies[['제목', '감독', '출연진', '장르']]

# 사용자 입력 받기
preferred_genre = input("선호하는 장르를 입력하세요: ")
num_recommendations = int(input("추천 받을 영화 수를 입력하세요: "))

# 영화 추천
recommendations = recommend_movies(preferred_genre, num_recommendations)

print("\n추천 영화:")
for _, movie in recommendations.iterrows():
    print(f"제목: {movie['제목']}")
    print(f"감독: {movie['감독']}")
    print(f"출연진: {movie['출연진']}")
    print(f"장르: {movie['장르']}")
    print()



추천 영화:
제목: 멍뭉이
감독: 김주환
출연진: 유연석, 차태현
장르: 드라마

제목: 안나푸르나
감독: 황승제
출연진: 김강현, 차선우, 한수연, 신연서, 서은채
장르: 드라마

제목: 그대 어이가리
감독: 이창열
출연진: 선동혁, 정아미, 박종진
장르: 드라마



# 영화 추천 방식
# 1. 사용자 입력: 사용자로부터 선호하는 장르, 배우, 감독 등의 정보를 입력받습니다.
# 2. 데이터 필터링: 입력받은 정보를 바탕으로 데이터프레임을 필터링합니다.
# 3. 점수 계산: 각 영화에 대해 사용자 선호도에 따른 점수를 계산합니다.
#    - 장르 일치: +3점
#    - 배우 일치: +2점 (주연배우 기준)
#    - 감독 일치: +2점
# 4. 정렬 및 추천: 계산된 점수를 기준으로 영화를 내림차순 정렬하고 상위 5편을 추천합니다.


In [166]:
def get_user_preferences():
    genre = input("선호하는 장르를 입력하세요: ")
    actor = input("선호하는 배우를 입력하세요: ")
    director = input("선호하는 감독을 입력하세요: ")
    return genre, actor, director

def calculate_score(row, genre, actor, director):
    score = 0
    if genre in row['장르']:
        score += 3
    if actor in row['출연진']:
        score += 2
    if director == row['감독']:
        score += 2
    return score

def recommend_movies(data, genre, actor, director):
    data['score'] = data.apply(lambda row: calculate_score(row, genre, actor, director), axis=1)
    recommended = data.sort_values('score', ascending=False).head(5)
    return recommended[['제목', '감독', '출연진', '장르', 'score']]

# 메인 실행 코드
genre, actor, director = get_user_preferences()
recommendations = recommend_movies(data, genre, actor, director)

print("\n추천 영화:")
print(recommendations.to_string(index=False))


추천 영화:
             제목  감독                               출연진              장르  score
     별 볼일 없는 인생 서동현                               정가은          멜로/로맨스      3
우리 사랑이 향기로 남을 때 임성용      윤시윤, 설인아, 노상현, 문지인, 이규복, 김영웅      멜로·로맨스·코미디      3
       10일간의 애인 이영용                               송민경          멜로/로맨스      3
        여섯 개의 밤 최창환      강길우, 강진아, 김시은, 변중희, 이한주, 정수지 드라마, 멜로·로맨스, 가족      3
       여덟 번째 감각 배인우 오준택, 임지섭, 박해인, 장영준, 방진원, 서지안, 채수아          멜로·로맨스      3


In [167]:
import pandas as pd
import numpy as np
from datetime import datetime

# 원본 데이터 로드
data = pd.read_csv('prob-0101.csv')

# 영화 추천 시스템 구현
def recommend_movies(data, user_preferences):
    """
    영화 추천 시스템
    
    추천 방식:
    1. 개봉일: 최신 영화에 더 높은 점수 부여
    2. 장르: 사용자 선호 장르와 일치하는 영화에 높은 점수 부여
    3. 출연진: 사용자가 선호하는 배우가 출연한 영화에 추가 점수 부여
    4. 감독: 사용자가 선호하는 감독의 영화에 추가 점수 부여
    """
    
    # 새로운 DataFrame 생성
    df = data.copy()
    
    # 현재 날짜
    current_date = datetime.now()
    
    # 개봉일 기반 점수 계산
    df['개봉일'] = pd.to_datetime(df['개봉일'])
    df['date_score'] = (current_date - df['개봉일']).dt.days
    df['date_score'] = 1 - (df['date_score'] - df['date_score'].min()) / (df['date_score'].max() - df['date_score'].min())
    
    # 장르 기반 점수 계산
    df['genre_score'] = df['장르'].apply(lambda x: any(genre in x for genre in user_preferences['genres']) if isinstance(x, str) else False).astype(int)
    
    # 출연진 기반 점수 계산
    df['cast_score'] = df['출연진'].apply(lambda x: sum(actor in x for actor in user_preferences['actors']) if isinstance(x, str) else 0)
    df['cast_score'] = df['cast_score'] / df['cast_score'].max() if df['cast_score'].max() > 0 else 0
    
    # 감독 기반 점수 계산
    df['director_score'] = df['감독'].apply(lambda x: 1 if x in user_preferences['directors'] else 0)
    
    # 최종 점수 계산
    df['score'] = (df['date_score'] * 0.3 + 
                   df['genre_score'] * 0.3 + 
                   df['cast_score'] * 0.2 + 
                   df['director_score'] * 0.2)
    
    # 불필요한 열 제거
    df = df.drop(['date_score', 'genre_score', 'cast_score', 'director_score'], axis=1)
    
    # 점수 기준 정렬
    df = df.sort_values('score', ascending=False).reset_index(drop=True)
    
    return df

# 사용자 선호도 예시
user_preferences = {
    'genres': ['드라마', '액션'],
    'actors': ['이하늬', '마동석'],
    'directors': ['이병헌', '박훈정']
}

# 영화 추천 실행
recommended_movies = recommend_movies(data, user_preferences)

# 결과 저장
recommended_movies.to_csv('recommended_movies.csv', index=False, encoding='utf-8-sig')

print("영화 추천이 완료되었습니다. 결과는 'recommended_movies.csv' 파일에 저장되었습니다.")


영화 추천이 완료되었습니다. 결과는 'recommended_movies.csv' 파일에 저장되었습니다.
